# TRẦN ĐỖ TRUNG KIÊN – 52000679
# HUỲNH HỒNG SƠN - 52000795
# NGUYỄN THÀNH HUY - 52100895

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import tensorflow as tf

# Thuật toán RNN trong Câu 3 được tích hợp cả phần Optimize hiệu năng

# 1. Thêm vào các hidden layer

# 2. Tham chiếu kết quả val_loss để từ đó điều chỉnh learning rate

# 3. Train the model với early stopping và hẹn lịch cho việc điều chỉnh learning rate

In [4]:
data = pd.read_csv('../data/diabetes_012_health_indicators_BRFSS2021.csv')

# Display first few rows of the dataset
print(data.head())

# Check for missing values
print(data.isnull().sum())

   Diabetes_012  HighBP  HighChol  CholCheck   BMI  Smoker  Stroke  \
0           0.0       0       1.0          1  15.0     1.0     0.0   
1           2.0       1       0.0          1  28.0     0.0     0.0   
2           2.0       1       1.0          1  33.0     0.0     0.0   
3           2.0       0       1.0          1  29.0     0.0     1.0   
4           0.0       0       0.0          1  24.0     1.0     0.0   

   HeartDiseaseorAttack  PhysActivity  Fruits  ...  AnyHealthcare  \
0                   0.0             0       1  ...              1   
1                   1.0             0       1  ...              1   
2                   0.0             1       1  ...              1   
3                   1.0             1       1  ...              1   
4                   0.0             0       0  ...              1   

   NoDocbcCost  GenHlth  MentHlth  PhysHlth  DiffWalk  Sex  Age  Education  \
0          0.0      5.0      10.0      20.0       0.0    0   11        4.0   
1       

In [5]:
# tach cot Benh ly tieu duong ra khoi df
X = data.drop('Diabetes_012', axis=1)
y = data['Diabetes_012']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# toi uu hoa LSTM
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

# chuyen thanh danh danh muc
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  # Assuming 2 classes for diabetes: 0 or 1
])

# model tuan tu
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50


KeyboardInterrupt: 

In [6]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")

# Predict from trained model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(np.array(y_test), axis=1)

# Report

# Độ chính xác hiện tại của

print(classification_report(y_true, y_pred_classes))

1478/1478 [==============================] - 2s 1ms/step - loss: 0.4179 - accuracy: 0.8411
Test Accuracy: 84.11%
1478/1478 [==============================] - 2s 1ms/step
              precision    recall  f1-score   support

           0       0.85      0.98      0.91     39508
           1       0.00      0.00      0.00      1079
           2       0.56      0.14      0.22      6689

    accuracy                           0.84     47276
   macro avg       0.47      0.37      0.38     47276
weighted avg       0.79      0.84      0.79     47276



c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:

# Thêm vào một hidden layer 256 neurons
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.LSTM(128, return_sequences=False),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(3, activation='softmax')  
])

# tham chiếu kết quả val_loss để từ đó điều chỉnh learning rate
lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model với early stopping và hẹn lịch cho việc điều chỉnh learning rate
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, batch_size=32, 
                    validation_split=0.2, verbose=1, 
                    callbacks=[early_stopping, lr_schedule])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy: {accuracy*100:.2f}%")




Epoch 1/100


4728/4728 [==============================] - 25s 5ms/step - loss: 0.4495 - accuracy: 0.8344 - val_loss: 0.4300 - val_accuracy: 0.8375 - lr: 0.0010
Epoch 2/100
4728/4728 [==============================] - 22s 5ms/step - loss: 0.4372 - accuracy: 0.8362 - val_loss: 0.4288 - val_accuracy: 0.8389 - lr: 0.0010
Epoch 3/100
4728/4728 [==============================] - 24s 5ms/step - loss: 0.4348 - accuracy: 0.8366 - val_loss: 0.4271 - val_accuracy: 0.8404 - lr: 0.0010
Epoch 4/100
4728/4728 [==============================] - 21s 4ms/step - loss: 0.4328 - accuracy: 0.8376 - val_loss: 0.4281 - val_accuracy: 0.8399 - lr: 0.0010
Epoch 5/100
4728/4728 [==============================] - 22s 5ms/step - loss: 0.4319 - accuracy: 0.8374 - val_loss: 0.4263 - val_accuracy: 0.8412 - lr: 0.0010
Epoch 6/100
4728/4728 [==============================] - 22s 5ms/step - loss: 0.4314 - accuracy: 0.8380 - val_loss: 0.4267 - val_accuracy: 0.8391 - lr: 0.0010
Epoch 7/100
4728/4728 [====================

# Kết quả thu về là 84.2%, không quá khác biệt với điểm số accuracy phía trên, là 84,13%